In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# SparkSessionの開始
spark = SparkSession.builder.appName("PredictionJob").getOrCreate()

# Delta Lake形式の新しいデータをS3から読み込み
new_data_delta_path = "/Volumes/databricks_test_ws/main/test-volume/data/new/diabetes"
new_data_spark_df = spark.read.format("delta").load(new_data_delta_path)

# Spark DataFrameをPandas DataFrameに変換
new_data_pdf = new_data_spark_df.toPandas()

# 特徴量の抽出
X_new = new_data_pdf.drop("target", axis=1, errors='ignore')  # 予測対象のデータにはtargetがない可能性があるのでerrors='ignore'を追加

# MLflowに保存されたモデルをロード
model_uri = "models:/random-forest-model/Production"  # モデルのURIを指定
rf_model = mlflow.sklearn.load_model(model_uri)

# 新しいデータに対する予測
predictions = rf_model.predict(X_new)

# 予測結果を保存
new_data_pdf['predictions'] = predictions
predictions_delta_path = "/Volumes/databricks_test_ws/main/test-volume/data/predictions/diabetes"
predictions_spark_df = spark.createDataFrame(new_data_pdf)
predictions_spark_df.write.format("delta").mode("overwrite").save(predictions_delta_path)

# 予測結果を表示
predictions_spark_df.show()
